<a href="https://colab.research.google.com/github/kimheeseo/2022_AI_SW_Programmingstudy/blob/main/EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# libraries
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
###########################################################
# EDA
###########################################################

df = pd.read_csv('data01_iris.csv') #데이터 불러오기, csv 파일은 데이터 나타낼 때 많이 사용하는 형식, excel로 실행 가능
df.head()
df.describe(include='all')
df['Species'].value_counts()

setosa        50
versicolor    50
virginica     50
Name: Species, dtype: int64

In [4]:
df #csv파일 잘 불러왔는 지 확인 가능(150*5 : 행렬 생성)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


In [5]:
df.info() #speices는 object인데, 이를 통해, data들의 type까지 파악 가능.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


In [6]:
df.describe() #데이터에 대한 분석 가능. 수치형 데이터에 대한 요약을 보여준다. std : 표준편차

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [9]:
df['Species'] #범주형 데이터

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: Species, Length: 150, dtype: object

In [ ]:
# visualization of pandas
df.hist('Sepal.Length')
df.boxplot('Sepal.Length')
df.plot.scatter('Sepal.Length','Sepal.Width')
SpeciesCode = df['Species'].astype('category').cat.codes
df.plot.scatter('Sepal.Length','Sepal.Width',c=SpeciesCode,colormap='viridis')


###########################################################
# Outlier Detection
###########################################################

# univariate outlier detection
# outlier check for each variable
df.boxplot('Sepal.Width')

# multivariate outlier detection
x = df.iloc[:,:-1].values
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
km.fit(x)
center_matrix = km.cluster_centers_[km.labels_,:]
dist = ((x-center_matrix)**2).sum(axis=1)
np.sort(dist)
outlier_idx = dist>2.0

df.plot.scatter('Sepal.Length','Sepal.Width',c=outlier_idx,colormap='viridis')
df.plot.scatter('Petal.Length','Petal.Width',c=outlier_idx,colormap='viridis')
df.plot.scatter('Sepal.Length','Petal.Length',c=outlier_idx,colormap='viridis')

# pca
from sklearn.decomposition import PCA
pca = PCA()
xtrans = pca.fit_transform(x)
df_trans = pd.DataFrame(xtrans)
df_trans.plot.scatter(0,1,c=outlier_idx,colormap='viridis')

# using sophisticated functions
from sklearn.neighbors import LocalOutlierFactor
f = LocalOutlierFactor()
detect = f.fit_predict(x)
outlier_idx2 = detect==-1
pd.crosstab(outlier_idx,outlier_idx2)
df_trans.plot.scatter(0,1,c=outlier_idx2,colormap='viridis')


###########################################################
# Missing Value Imputation 
###########################################################

df = pd.read_csv('data11_khan2.csv')
xorg = df.values

# missing value detection
naidx = np.isnan(xorg)
naidx.sum()
naidx.sum(axis=0)
naidx.sum(axis=1)
xorg[naidx]

# simple missing value imputation
from sklearn.impute import SimpleImputer

f = SimpleImputer(missing_values=np.nan,strategy='mean')
xfill = f.fit_transform(xorg)
np.isnan(xfill).sum().sum()
xfill[naidx]

f = SimpleImputer(missing_values=np.nan,strategy='constant',fill_value=0)
xfill = f.fit_transform(xorg)
xfill[naidx]

